Procesamiento de datos usando engine y Pandas
===

* Última modificación: 22 de septiembre de 2022

Este tutorial esta basado en https://es.hortonworks.com/tutorial/how-to-process-data-with-apache-hive/

Fuentes de datos
---

In [16]:
filenames = [
    "drivers.csv",
    "timesheet.csv",
    "truck_event_text_partition.csv",
]

url = "https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/drivers/"

for filename in filenames:
    !curl --silent -o /tmp/{filename} {url + filename}
    
!ls -1 /tmp/*.csv

/tmp/drivers.csv
/tmp/timesheet.csv
/tmp/truck_event_text_partition.csv


In [17]:
#
# Se imprime el final del archivo drivers
#
!tail -n 5 /tmp/drivers.csv

39,David Kaiser,967706052,9185 At Street,Y,hours
40,Nicolas Maillard,208510217,1027 Quis Rd.,Y,hours
41,Greg Phillips,308103116,P.O. Box 847- 5961 Arcu. Road,Y,hours
42,Randy Gelhausen,853302254,145-4200 In- Avenue,Y,hours
43,Dave Patton,977706052,3028 A- St.,Y,hours

In [18]:
!tail -n 5 /tmp/timesheet.csv

43,48,52,2517
43,49,56,2743
43,50,59,2665
43,51,58,2593
43,52,48,2764

Creación de la máquina
---

In [19]:
from sqlalchemy import create_engine

engine = create_engine(
    "sqlite+pysqlite:///:memory:",
    echo=False,
    future=True,
)

Creación y carga de datos de datos para la tabla `drivers` usando pandas
---

In [20]:
import pandas as pd

drivers = pd.read_csv(
    "/tmp/drivers.csv",
    sep=",",
    thousands=None,
    decimal=".",
    encoding="latin-1",
)

drivers.columns = [col.replace("-", "_") for col in drivers.columns]

drivers.to_sql(
    "drivers",
    con=engine,
    if_exists="replace",
    index=False,
)

34

In [21]:
#
# Verificación
#
from sqlalchemy.sql import text

sql = """
    SELECT * FROM drivers LIMIT 5;
"""
with engine.connect().execution_options(autocommit=True) as conn:
    query = conn.execute(text(sql))

pd.DataFrame(query.fetchall())

,driverId,name,ssn,location,certified,wage_plan
0,10,George Vetticaden,621011971,244-4532 Nulla Rd.,N,miles
1,11,Jamie Engesser,262112338,366-4125 Ac Street,N,miles
2,12,Paul Coddin,198041975,Ap #622-957 Risus. Street,Y,hours
3,13,Joe Niemiec,139907145,2071 Hendrerit. Ave,Y,hours
4,14,Adis Cesir,820812209,Ap #810-1228 In St.,Y,hours


Extracción de datos de la DB con Pandas
----

In [22]:
query = "SELECT * FROM drivers LIMIT 5;"

with engine.connect().execution_options(autocommit=True) as conn:
    print(pd.read_sql_query(text(query), conn))

   driverId               name        ssn                   location  \
0        10  George Vetticaden  621011971         244-4532 Nulla Rd.   
1        11     Jamie Engesser  262112338         366-4125 Ac Street   
2        12        Paul Coddin  198041975  Ap #622-957 Risus. Street   
3        13        Joe Niemiec  139907145        2071 Hendrerit. Ave   
4        14         Adis Cesir  820812209        Ap #810-1228 In St.   

  certified wage_plan  
0         N     miles  
1         N     miles  
2         Y     hours  
3         Y     hours  
4         Y     hours  


In [23]:
with engine.connect().execution_options(autocommit=True) as conn:
    print(pd.read_sql_table("drivers", conn))

    driverId                 name        ssn  \
0         10    George Vetticaden  621011971   
1         11       Jamie Engesser  262112338   
2         12          Paul Coddin  198041975   
3         13          Joe Niemiec  139907145   
4         14           Adis Cesir  820812209   
5         15         Rohit Bakshi  239005227   
6         16           Tom McCuch  363303105   
7         17          Eric Mizell  123808238   
8         18            Grant Liu  171010151   
9         19           Ajay Singh  160005158   
10        20         Chris Harris  921812303   
11        21         Jeff Markham  209408086   
12        22        Nadeem Asghar  783204269   
13        23            Adam Diaz  928312208   
14        24          Don Hilborn  254412152   
15        25  Jean-Philippe Playe  913310051   
16        26         Michael Aube  124705141   
17        27      Mark Lochbihler  392603159   
18        28      Olivier Renault  959908181   
19        29           Teddy Choi  18550

Creación y carga de datos de la tabla `timesheet`
---

In [24]:
timesheet = pd.read_csv(
    "/tmp/timesheet.csv",
    sep=",",
    thousands=None,
    decimal=".",
    encoding="latin-1",
)

timesheet.columns = [col.replace("-", "_") for col in timesheet.columns]

timesheet.to_sql(
    "timesheet",
    con=engine,
    if_exists="replace",
    index=False,
)

1768

In [25]:
#
# Verificación
#
sql = """
    SELECT * FROM timesheet LIMIT 5;
"""
with engine.connect().execution_options(autocommit=True) as conn:
    query = conn.execute(text(sql))

pd.DataFrame(query.fetchall())

,driverId,week,hours_logged,miles_logged
0,10,1,70,3300
1,10,2,70,3300
2,10,3,60,2800
3,10,4,70,3100
4,10,5,70,3200


Cantidad de horas y millas de cada conductor por año.
----

In [26]:
sql = """
SELECT 
    driverId, 
    sum(hours_logged), 
    sum(miles_logged) 
FROM 
    timesheet 
GROUP BY 
    driverId;
"""
with engine.connect().execution_options(autocommit=True) as conn:
    query = conn.execute(text(sql))

pd.DataFrame(query.fetchall())

,driverId,sum(hours_logged),sum(miles_logged)
0,10,3232,147150
1,11,3642,179300
2,12,2639,135962
3,13,2727,134126
4,14,2781,136624
5,15,2734,138750
6,16,2746,137205
7,17,2701,135992
8,18,2654,137834
9,19,2738,137968


Subconsultas
---

In [27]:
#
# Selecciona todos los registros de la tabla timesheet
# que tienen el driverId igual al menor driverId de la
# tabla drivers.
#
sql = """
SELECT 
    *
FROM
    timesheet
WHERE
    driverId = (SELECT min(driverId) FROM drivers);
"""
with engine.connect().execution_options(autocommit=True) as conn:
    query = conn.execute(text(sql))

pd.DataFrame(query.fetchall())

,driverId,week,hours_logged,miles_logged
0,10,1,70,3300
1,10,2,70,3300
2,10,3,60,2800
3,10,4,70,3100
4,10,5,70,3200
5,10,6,70,3300
6,10,7,70,3000
7,10,8,70,3300
8,10,9,70,3200
9,10,10,50,2500


Consulta para unir las tablas
---

In [28]:
sql = """
SELECT 
    d.driverId, 
    d.name, 
    t.total_hours, 
    t.total_miles 
FROM 
    drivers d
JOIN (
    SELECT 
        driverId, 
        sum(hours_logged)total_hours, 
        sum(miles_logged)total_miles 
    FROM 
        timesheet 
    GROUP BY 
        driverId 
    ) t
ON 
    (d.driverId = t.driverId);
"""
with engine.connect().execution_options(autocommit=True) as conn:
    query = conn.execute(text(sql))

pd.DataFrame(query.fetchall())

,driverId,name,total_hours,total_miles
0,10,George Vetticaden,3232,147150
1,11,Jamie Engesser,3642,179300
2,12,Paul Coddin,2639,135962
3,13,Joe Niemiec,2727,134126
4,14,Adis Cesir,2781,136624
5,15,Rohit Bakshi,2734,138750
6,16,Tom McCuch,2746,137205
7,17,Eric Mizell,2701,135992
8,18,Grant Liu,2654,137834
9,19,Ajay Singh,2738,137968


In [29]:
#
# Conductor con más horas registradas
#
sql = """
SELECT 
    d.driverId, 
    d.name, 
    t.total_hours, 
    t.total_miles 
FROM 
    drivers d
JOIN (
    SELECT 
        driverId, 
        sum(hours_logged)total_hours, 
        sum(miles_logged)total_miles 
    FROM 
        timesheet 
    GROUP BY 
        driverId 
    ) t
ON 
    (d.driverId = t.driverId)
ORDER BY
    t.total_hours DESC
LIMIT 1;
"""

with engine.connect().execution_options(autocommit=True) as conn:
    query = conn.execute(text(sql))

pd.DataFrame(query.fetchall())

,driverId,name,total_hours,total_miles
0,11,Jamie Engesser,3642,179300


Creación de un DataFrame usando una consulta
---

In [30]:
sql = """
SELECT 
    d.driverId, 
    d.name, 
    t.total_hours, 
    t.total_miles 
FROM 
    drivers d
JOIN (
    SELECT 
        driverId, 
        sum(hours_logged) total_hours, 
        sum(miles_logged) total_miles 
    FROM 
        timesheet 
    GROUP BY 
        driverId 
    ) t
ON 
    (d.driverId = t.driverId)
ORDER BY
    t.total_hours DESC;
"""

with engine.connect().execution_options(autocommit=True) as conn:
    query = conn.execute(text(sql))

df = pd.DataFrame(query.fetchall())


import matplotlib as mpl
import matplotlib.pyplot as plt

df.plot.bar(x="name", y="total_hours");

ModuleNotFoundError: No module named 'matplotlib'

Almacenamiento de los resultados
---

In [ ]:
sql = """
SELECT 
    d.driverId, 
    d.name, 
    t.total_hours, 
    t.total_miles 
FROM 
    drivers d
JOIN (
    SELECT 
        driverId, 
        sum(hours_logged) total_hours, 
        sum(miles_logged) total_miles 
    FROM 
        timesheet 
    GROUP BY 
        driverId 
    ) t
ON 
    (d.driverId = t.driverId)
ORDER BY
    d.name;
"""

with engine.connect().execution_options(autocommit=True) as conn:
    query = conn.execute(text(sql))

pd.DataFrame(query.fetchall()).to_csv("/tmp/summary.csv", index=False)

In [ ]:
!head /tmp/summary.csv

driverId,name,total_hours,total_miles
23,Adam Diaz,2750,137980
14,Adis Cesir,2781,136624
19,Ajay Singh,2738,137968
36,Andrew Grande,2795,138025
20,Chris Harris,2644,134564
30,Dan Rice,2773,137473
43,Dave Patton,2750,136993
39,David Kaiser,2745,138788
24,Don Hilborn,2647,134461
